In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

In [15]:
initial_guess = pd.read_csv("output/savgol.csv")
initial_guess

,timestamp,underlying,expiry,call_iv_23500,call_iv_23600,call_iv_23700,call_iv_23800,call_iv_23900,call_iv_24000,call_iv_24100,...,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41
0,1746590742000000000,24433.0,2025-05-08,0.435793,0.409610,0.383775,0.356050,0.322957,0.288892,0.254962,...,-0.022656,0.006262,2.553592e+06,-0.001931,2.372125,0.068108,-1.522534e+06,-3.048516e+06,1.254699e+06,-0.000000e+00
1,1746176782000000000,24285.5,2025-05-08,0.205534,0.196490,0.187751,0.178467,0.170073,0.161772,0.154975,...,0.003836,-0.004734,6.693594e+05,0.010055,1.315715,0.030237,5.235704e+04,-8.124241e+06,1.313372e+06,-1.000000e-06
2,1746520665000000000,24387.5,2025-05-08,0.290264,0.270770,0.247515,0.224138,0.199536,0.173109,0.149435,...,0.001037,0.003309,2.847738e+06,0.008796,0.648142,0.041674,3.459928e+05,-2.764208e+07,2.203250e+06,0.000000e+00
3,1746591359000000000,24384.6,2025-05-08,0.426385,0.396069,0.369804,0.341066,0.309390,0.276375,0.242813,...,0.035097,-0.030470,-3.629051e+06,0.146150,-8.941554,1.278703,6.211531e+06,7.623811e+06,-5.587995e+05,3.208558e+06
4,1746165957000000000,24329.3,2025-05-08,0.211338,0.203834,0.196625,0.188698,0.181008,0.173976,0.167036,...,0.001900,0.002318,-1.051215e+06,0.026776,0.242156,0.025405,2.716713e+06,-3.698245e+06,-0.000000e+00,-0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23748,1746591740000000000,24335.0,2025-05-08,0.406088,0.377406,0.349971,0.320558,0.289179,0.256396,0.221398,...,0.008603,-0.004291,-1.986960e+06,0.012081,-1.567061,0.025376,-1.849046e+05,9.476678e+05,-8.574375e+05,-0.000000e+00
23749,1746517365000000000,24379.3,2025-05-08,0.281599,0.263384,0.241601,0.219036,0.195533,0.172074,0.149054,...,-0.009330,0.001730,3.085312e+05,-0.010100,0.017339,0.041731,-5.464379e+06,9.938785e+06,0.000000e+00,0.000000e+00
23750,1746510841000000000,24383.8,2025-05-08,0.277396,0.256449,0.235455,0.214142,0.191823,0.168970,0.146202,...,-0.006017,-0.014316,-5.171651e+05,0.009618,-0.905712,-0.035879,-5.117804e+05,-3.953448e+07,1.914297e+05,-0.000000e+00
23751,1746603610000000000,24348.2,2025-05-08,0.386717,0.355572,0.323200,0.296829,0.262362,0.226321,0.190997,...,0.001378,0.003366,1.357577e+06,0.016415,0.766966,-0.016124,-9.293144e+05,6.414991e+06,3.323359e+04,0.000000e+00


In [16]:
test_orig = pd.read_parquet("sample_val.parquet")
test_orig

,timestamp,underlying,expiry,call_iv_23500,call_iv_23600,call_iv_23700,call_iv_23800,call_iv_23900,call_iv_24000,call_iv_24100,...,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41
158881,1746590742000000000,24433.0,2025-05-08,0.435793,0.409610,0.383775,0.356050,NaN,NaN,NaN,...,-0.022656,0.006262,2.553592e+06,-0.001931,2.372125,0.068108,-1.522534e+06,-3.048516e+06,1.254699e+06,-0.000000e+00
117518,1746176782000000000,24285.5,2025-05-08,NaN,NaN,0.187751,0.178467,0.170073,0.161772,0.154975,...,0.003836,-0.004734,6.693594e+05,0.010055,1.315715,0.030237,5.235704e+04,-8.124241e+06,1.313372e+06,-1.000000e-06
155403,1746520665000000000,24387.5,2025-05-08,NaN,0.270770,NaN,NaN,NaN,0.173109,NaN,...,0.001037,0.003309,2.847738e+06,0.008796,0.648142,0.041674,3.459928e+05,-2.764208e+07,2.203250e+06,0.000000e+00
159498,1746591359000000000,24384.6,2025-05-08,0.426385,0.396069,NaN,0.341066,NaN,NaN,0.242813,...,0.035097,-0.030470,-3.629051e+06,0.146150,-8.941554,1.278703,6.211531e+06,7.623811e+06,-5.587995e+05,3.208558e+06
106693,1746165957000000000,24329.3,2025-05-08,NaN,NaN,0.196625,NaN,NaN,NaN,NaN,...,0.001900,0.002318,-1.051215e+06,0.026776,0.242156,0.025405,2.716713e+06,-3.698245e+06,-0.000000e+00,-0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159879,1746591740000000000,24335.0,2025-05-08,NaN,0.377406,0.349971,NaN,NaN,0.256396,NaN,...,0.008603,-0.004291,-1.986960e+06,0.012081,-1.567061,0.025376,-1.849046e+05,9.476678e+05,-8.574375e+05,-0.000000e+00
152103,1746517365000000000,24379.3,2025-05-08,0.281599,0.263384,0.241601,NaN,NaN,NaN,0.149054,...,-0.009330,0.001730,3.085312e+05,-0.010100,0.017339,0.041731,-5.464379e+06,9.938785e+06,0.000000e+00,0.000000e+00
145579,1746510841000000000,24383.8,2025-05-08,0.277396,0.256449,NaN,0.214142,0.191823,0.168970,0.146202,...,-0.006017,-0.014316,-5.171651e+05,0.009618,-0.905712,-0.035879,-5.117804e+05,-3.953448e+07,1.914297e+05,-0.000000e+00
171749,1746603610000000000,24348.2,2025-05-08,0.386717,0.355572,0.323200,0.296829,NaN,NaN,0.190997,...,0.001378,0.003366,1.357577e+06,0.016415,0.766966,-0.016124,-9.293144e+05,6.414991e+06,3.323359e+04,0.000000e+00


In [17]:
val_Y = pd.read_parquet("val_Y.parquet")
sample_val = pd.read_parquet("sample_val.parquet")
sample_val_matching = pd.read_csv("output/matching.csv")

In [18]:
pred_cols = [col for col in val_Y.columns if "iv_" in col]

In [19]:
test_initial = val_Y.copy()
test_initial[initial_guess.columns] = initial_guess.values
for col in pred_cols:
    test_initial[col] = test_initial[col].astype(np.float64)
test_initial

,timestamp,underlying,expiry,call_iv_23500,call_iv_23600,call_iv_23700,call_iv_23800,call_iv_23900,call_iv_24000,call_iv_24100,...,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41
158881,1746590742000000000,24433.0,2025-05-08,0.435793,0.409610,0.383775,0.356050,0.322957,0.288892,0.254962,...,-0.022656,0.006262,2553592.317708,-0.001931,2.372125,0.068108,-1522534.267609,-3048515.625002,1254698.567708,-0.0
117518,1746176782000000000,24285.5,2025-05-08,0.205534,0.196490,0.187751,0.178467,0.170073,0.161772,0.154975,...,0.003836,-0.004734,669359.375,0.010055,1.315715,0.030237,52357.037396,-8124241.450085,1313372.395833,-0.000001
155403,1746520665000000000,24387.5,2025-05-08,0.290264,0.270770,0.247515,0.224138,0.199536,0.173109,0.149435,...,0.001037,0.003309,2847737.5,0.008796,0.648142,0.041674,345992.750162,-27642078.409655,2203250.0,0.0
159498,1746591359000000000,24384.6,2025-05-08,0.426385,0.396069,0.369804,0.341066,0.309390,0.276375,0.242813,...,0.035097,-0.03047,-3629051.116071,0.14615,-8.941554,1.278703,6211531.158559,7623811.4435,-558799.479167,3208557.54636
106693,1746165957000000000,24329.3,2025-05-08,0.211338,0.203834,0.196625,0.188698,0.181008,0.173976,0.167036,...,0.0019,0.002318,-1051214.558532,0.026776,0.242156,0.025405,2716712.748023,-3698244.79213,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159879,1746591740000000000,24335.0,2025-05-08,0.406088,0.377406,0.349971,0.320558,0.289179,0.256396,0.221398,...,0.008603,-0.004291,-1986960.416667,0.012081,-1.567061,0.025376,-184904.558875,947667.825668,-857437.5,-0.0
152103,1746517365000000000,24379.3,2025-05-08,0.281599,0.263384,0.241601,0.219036,0.195533,0.172074,0.149054,...,-0.00933,0.00173,308531.25,-0.0101,0.017339,0.041731,-5464379.197581,9938785.177713,0.0,0.0
145579,1746510841000000000,24383.8,2025-05-08,0.277396,0.256449,0.235455,0.214142,0.191823,0.168970,0.146202,...,-0.006017,-0.014316,-517165.104167,0.009618,-0.905712,-0.035879,-511780.361459,-39534479.448244,191429.6875,-0.0
171749,1746603610000000000,24348.2,2025-05-08,0.386717,0.355572,0.323200,0.296829,0.262362,0.226321,0.190997,...,0.001378,0.003366,1357577.34375,0.016415,0.766966,-0.016124,-929314.409647,6414991.456032,33233.59375,0.0


In [20]:
test_initial.isna().sum().sum()

0

In [21]:
from sklearn.metrics import root_mean_squared_error as rmse

rmse(val_Y[pred_cols], test_initial[pred_cols])

0.0008293302951223861

# NOTE CHANGING TEST_INITIAL BELOW

In [22]:
initial_guess = pd.read_csv("../data/savgol.csv")
test_orig = pd.read_parquet("../data/test_data.parquet")
pred_cols = [col for col in test_orig.columns if "iv_" in col]
test_initial = test_orig.copy()
test_initial[initial_guess.columns] = initial_guess.values
for col in pred_cols:
    test_initial[col] = test_initial[col].astype(np.float64)
test_initial

,timestamp,underlying,call_iv_24000,call_iv_24100,call_iv_24200,call_iv_24300,call_iv_24400,call_iv_24500,call_iv_24600,call_iv_24700,...,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41
0,0.0,24735.9,0.280939,0.261703,0.251835,0.251139,0.242149,0.238177,0.232439,0.226372,...,0.006587,0.002826,-1.737617e+05,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06
1,1.0,24766.7,0.270276,0.267392,0.258893,0.249978,0.244150,0.239080,0.233548,0.228081,...,0.005777,0.004588,-3.195086e+05,-0.024106,-0.004696,-0.000158,-4.529075e+06,-1.619022e+06,-0.956928,4.624907e+06
2,2.0,24896.9,0.257849,0.251731,0.237410,0.224546,0.214869,0.204580,0.194604,0.187329,...,0.000829,-0.034944,1.027525e+06,0.028201,0.032234,0.007687,2.663908e+06,0.000000e+00,-0.116264,-2.669766e+06
3,3.0,24898.1,0.241888,0.231807,0.220505,0.209565,0.198602,0.186190,0.174505,0.165883,...,-0.009323,-0.022969,-4.720074e+06,-0.001513,0.008704,-0.000206,-7.672936e+06,-1.903406e+06,-2.249208,-1.858254e+07
4,4.0,24906.5,0.235328,0.229625,0.222983,0.214126,0.206515,0.199258,0.192603,0.187139,...,-0.018674,-0.007588,-4.051681e+05,-0.136267,0.002425,-0.002200,1.334469e+06,2.548789e+05,1.999104,3.024212e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12060,12060.0,24992.1,0.243403,0.233427,0.224683,0.215809,0.207159,0.199425,0.192689,0.186856,...,-0.010750,0.006911,-1.497117e+05,-0.059785,-0.014518,-0.006756,-7.822864e+06,2.149933e+05,-2.069687,-6.738094e+06
12061,12061.0,25020.3,0.241702,0.236380,0.226972,0.217590,0.209007,0.200906,0.193655,0.187869,...,-0.000224,0.024468,-1.176289e+06,-0.016344,-0.034368,0.001133,-2.392407e+06,-1.176289e+06,-2.023157,6.031379e+03
12062,12062.0,24897.2,0.256475,0.243817,0.234267,0.221270,0.211324,0.201792,0.192578,0.185088,...,-0.009009,0.006057,1.920387e+06,-0.016521,-0.013230,-0.009278,2.287786e+06,0.000000e+00,1.751653,-3.621450e+06
12063,12063.0,25035.6,0.249808,0.239825,0.231439,0.221770,0.212413,0.204499,0.197813,0.190833,...,0.001625,0.063421,6.357844e+05,0.077753,0.068983,0.000112,8.723009e+06,2.352188e+05,1.502687,-1.177899e+06


In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from xgboost import XGBRegressor


def xgb_mice_imputation(test_df, max_iter=10, tol=1e-4):
    """Enhanced iterative XGBoost imputation for datasets with no complete rows."""

    # Identify features and IV columns
    feature_cols = [f"X{i}" for i in range(42) if f"X{i}" in test_df.columns] + [
        "underlying"
    ]
    iv_cols = [col for col in test_df.columns if "iv_" in col]

    test_imputed = test_df.copy()

    # ---- Better Initial Imputation: Use KNNImputer ----
    initial_imputer = KNNImputer(n_neighbors=5)
    all_cols = feature_cols + iv_cols
    test_imputed[all_cols] = initial_imputer.fit_transform(test_imputed[all_cols])

    # ---- Normalize Features Only (not IVs) ----
    scaler = StandardScaler()
    test_imputed[feature_cols] = scaler.fit_transform(test_imputed[feature_cols])

    prev_imputed = test_imputed[iv_cols].copy()

    for iteration in range(max_iter):
        print(f"Iteration {iteration + 1}/{max_iter}")
        np.random.shuffle(iv_cols)  # Randomize IV column order

        for target_col in iv_cols:
            missing_mask = test_df[target_col].isna()
            if not missing_mask.any():
                continue

            predictor_cols = [
                col for col in feature_cols + iv_cols if col != target_col
            ]

            X_train = test_imputed.loc[~missing_mask, predictor_cols]
            y_train = test_imputed.loc[~missing_mask, target_col]

            if len(X_train) < 10:
                median_val = y_train.median()
                test_imputed.loc[missing_mask, target_col] = median_val
                continue

            X_pred = test_imputed.loc[missing_mask, predictor_cols]

            # ---- More efficient and accurate model ----
            model = XGBRegressor(
                objective="reg:squarederror",
                n_estimators=200,
                max_depth=4,
                learning_rate=0.05,
                subsample=0.8,
                colsample_bytree=0.8,
                n_jobs=-1,
                verbosity=0,
            )
            model.fit(X_train, y_train)

            preds = model.predict(X_pred)
            test_imputed.loc[missing_mask, target_col] = np.clip(preds, 0.01, 2.0)

        # ---- Convergence check ----
        current_imputed = test_imputed[iv_cols]
        delta = (current_imputed - prev_imputed).abs().mean().mean()
        print(f"  Mean change: {delta:.6f}")

        if delta < tol:
            print(f"Converged after {iteration+1} iterations")
            break

        prev_imputed = current_imputed.copy()

    # Final clipping
    test_imputed[iv_cols] = test_imputed[iv_cols].clip(0.01, 2.0)

    return test_imputed

In [24]:
imputed = xgb_mice_imputation(test_orig, max_iter=5, tol=1e-4)

Iteration 1/5
  Mean change: 0.019029
Iteration 2/5
  Mean change: 0.000395
Iteration 3/5
  Mean change: 0.000141
Iteration 4/5
  Mean change: 0.000117
Iteration 5/5
  Mean change: 0.000104


In [28]:
sample_submission = pd.read_csv(f"../data/sample_submission.csv")
submission_cols = sample_submission.columns.tolist()

In [29]:
sub = imputed[submission_cols].copy()
sub.to_csv("output/xgboost.csv", index=False)
sub

,timestamp,call_iv_24000,call_iv_24100,call_iv_24200,call_iv_24300,call_iv_24400,call_iv_24500,call_iv_24600,call_iv_24700,call_iv_24800,...,put_iv_24600,put_iv_24700,put_iv_24800,put_iv_24900,put_iv_25000,put_iv_25100,put_iv_25200,put_iv_25300,put_iv_25400,put_iv_25500
0,0,0.280939,0.264128,0.256697,0.249115,0.242149,0.237653,0.232439,0.226144,0.222997,...,0.232334,0.226171,0.222378,0.227301,0.234169,0.244438,0.250422,0.258191,0.271874,0.282229
1,1,0.270276,0.268090,0.258893,0.250671,0.243970,0.239136,0.233548,0.228083,0.225120,...,0.234058,0.228209,0.224660,0.229871,0.238987,0.249402,0.258142,0.264941,0.271767,0.286263
2,2,0.256581,0.251731,0.236792,0.224783,0.214869,0.204580,0.194604,0.188302,0.183447,...,0.194612,0.188052,0.183156,0.180492,0.181346,0.185063,0.190750,0.196255,0.202708,0.210563
3,3,0.241888,0.231088,0.220505,0.208886,0.198602,0.186190,0.175080,0.166695,0.161899,...,0.175356,0.166394,0.161561,0.159613,0.163783,0.172032,0.181135,0.189797,0.197163,0.206107
4,4,0.235328,0.229777,0.222983,0.214126,0.205954,0.199071,0.192603,0.186284,0.181777,...,0.192783,0.186034,0.181701,0.177920,0.175537,0.176000,0.177829,0.178887,0.182314,0.186202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12060,12060,0.241971,0.233427,0.226425,0.216303,0.206775,0.199545,0.192691,0.186856,0.182053,...,0.192912,0.186668,0.182099,0.177270,0.175206,0.174541,0.176356,0.177716,0.180035,0.182888
12061,12061,0.243504,0.236380,0.226972,0.217421,0.208855,0.200906,0.193655,0.187764,0.183716,...,0.193816,0.187600,0.183554,0.180196,0.177969,0.177923,0.179519,0.180254,0.182133,0.184173
12062,12062,0.256475,0.243817,0.234267,0.221270,0.211193,0.201427,0.192355,0.185088,0.179760,...,0.192306,0.184825,0.179666,0.176916,0.177432,0.181574,0.187865,0.194213,0.201360,0.209940
12063,12063,0.248623,0.239825,0.231687,0.221655,0.213236,0.205346,0.197433,0.190833,0.185851,...,0.197734,0.190933,0.186172,0.181716,0.178414,0.177504,0.177803,0.178806,0.179977,0.182085


In [27]:
# from sklearn.metrics import root_mean_squared_error as rmse

# rmse(val_Y[pred_cols], imputed[pred_cols]) 